In [1]:
%reload_ext Cython
%reload_ext line_profiler

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket
import PyHEADTAIL.particles.generators as gen

PyHEADTAIL v1.9.3.12




### Create bunches and bunches_list

In [4]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 40
alpha = 3.225e-4
h1 = 20
V1 = 1000
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [5]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=circumference,
    alpha_array=[alpha], p_increment=0,
    harmonic_list=[h1], voltage_list=[V1], phi_offset_list=[0])

In [6]:
dt = np.random.uniform(0, 10e-9, 3)
epsn_z = np.random.uniform(0, 0.5, 3)
print dt
print epsn_z

[  7.16509355e-09   3.75595475e-09   9.55430869e-09]
[ 0.44193405  0.47084237  0.45004354]


In [7]:
T0 = circumference/beta/c
dt = (0, 2.5e-9, 7.5e-9, T0+0, T0+2.5e-9, T0+7.5e-9)
epsn_z = (0.35, 0.25, 0.1, 0.35, 0.25, 0.1)
# epsn_z = (0.1, 0.25, 0.35, 0.1, 0.25, 0.35)

In [8]:
bunches_list = [
    gen.ParticleGenerator(
        macroparticlenumber=1e5, intensity=1e11,
        charge=e, mass=m_p, gamma=gamma,
        circumference=circumference,
        distribution_x=gen.gaussian2D(2e-6),
        distribution_y=gen.gaussian2D(2e-6),
        distribution_z=gen.RF_bucket_distribution(rfbucket, epsn_z=epsn_z[i])
    ).generate() for i in xrange(len(dt))]

for i, b in enumerate(bunches_list):
    b.z -= dt[i] * (b.beta*c)
    b.dt = dt[i]

bunches = sum(bunches_list)
bunches.dt = dt[0]

*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 2.50e-02
... distance to target emittance: 2.51e-02
... distance to target emittance: -1.77e-04
--> Emittance: 0.350001068509
--> Bunch length:0.164249527414

/home/kli/workspace/PyHEADTAIL/particles/generators.py:330: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  coords = [np.random.normal(loc=0., scale=std, size=n_particles),
/home/kli/workspace/PyHEADTAIL/particles/generators.py:331: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  np.random.normal(loc=0., scale=std, size=n_particles)]
/home/kli/workspace/PyHEADTAIL/particles/generators.py:530: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  u = uniform(low=xmin, high=xmax, size=n_gen)



*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.35e-02
... distance to target emittance: 1.36e-02
... distance to target emittance: 2.72e-05
--> Emittance: 0.250000058984
--> Bunch length:0.137176493154
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.90e-03
... distance to target emittance: 2.02e-03
... distance to target emittance: 7.91e-07
--> Emittance: 0.100000000329
--> Bunch length:0.0853883539502
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 2.50e-02
... distance to target emittance: 2.51e-02
... distance to target emittance: -1.77e-04
--> Emittance: 0.350001068509
--> Bunch length:0.164249527414
*** Maximum RMS emittance 1.12145002894eV s.
... distance to target emittance: 1.35e-02
... distance to target emittance: 1.36e-02
... distance to target emittance: 2.72e-05
--> Emittance: 0.250000058984
--> Bunch length:0.137176493154
*** Maximum RMS emittance 1.12145002894eV s.
... dist

/home/kli/workspace/PyHEADTAIL/particles/generators.py:531: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v = uniform(low=ymin, high=ymax, size=n_gen)
/home/kli/workspace/PyHEADTAIL/particles/generators.py:532: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  s = uniform(size=n_gen)


### Wakes and wake kicks and functions

In [9]:
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import CircularResonator, WakeField

In [24]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

wake = CircularResonator(R_shunt=1e9, frequency=1e9, Q=500, n_turns_wake=2)

wakefields = WakeField(slicer, wake, circumference=circumference)
wakekick = wakefields.wake_kicks[0]
wakefunction = wakekick.wake_function

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


### Convolutions

In [25]:
def convolution_python(t_target, t_source, c_source, w):
    dxp = 0.*t_target
    for k in xrange(len(t_target)):
        for l in xrange(len(t_source)):
            dxp[k] += c_source[l]*w(t_target[k]-t_source[l])

    return dxp

In [26]:
def convolution_numpy(t_target, t_source, c_source, w):
    tmin, tmax = t_source[0], t_source[-1]
    tt = np.concatenate((t_target-tmax, (t_target-tmin)[1:]))
    
    return np.convolve(c_source, w(tt), mode='valid')

In [27]:
def convolve_multibunch(times, moments, wf, dt=None, f_convolution=convolution_python):
    dxp = []
    if dt is None: dt = 0.*np.array(times)
    for i in xrange(len(times)):
        z = 0.*times[i]
        t_target = times[i]
        for j in range(i+1):
            t_source = times[j] + dt[i] - dt[j]
            c_source = moments[j]
            z += f_convolution(t_target, t_source, c_source, wf)
            
        dxp.append(z)
        
    return dxp

In [28]:
def convolve_PyHEADTAIL_sb(bunch, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal(bunch, times, ages, cmoments, betas)

In [29]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

### Use cases
#### Create slices and slices_list
---

In [16]:
def get_times_and_moments(bunches_list, slicer):
    bunches_list = np.atleast_1d(bunches_list)

    slices_list = []
    times_list = []
    cmoments_list = []
    for i, b in enumerate(bunches_list):
        distance = dt[i]*b.beta*c
        b.z += distance
        slices_list.append(b.get_slices(slicer))
        times_list.append(slices_list[-1].z_centers / (b.beta*c))
        cmoments_list.append(slices_list[-1].charge_per_slice)
        b.z -= distance

    # Multi-turn multi-bunch times and moments arrays - here, only one turn assumed
    n_turns = 1
    n_bunches = len(bunches_list)
    n_slices = len(times_list[0])

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = times_list
    times_array = tmp

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = cmoments_list
    cmoments_array = tmp

    ages_list = np.array([0 for b in bunches_list])
    betas_list = np.array([[b.beta for b in bunches_list]])
    
    return times_array, ages_list, cmoments_array, betas_list

#### Single beam

In [17]:
# Single beam
slicer = UniformBinSlicer(2000, z_cuts=(-45, .5))

times, ages, cmoments, betas = get_times_and_moments(bunches, slicer)

delta_xp_mb = convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)

times_list = times[:,0,:]
cmoments_list = cmoments[:,0,:]

delta_xp_sb = convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)
delta_xp_np = convolve_multibunch(times_list, cmoments_list, wakefunction)
# %timeit convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction, f_convolution=convolution_numpy)

In [33]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 6)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments))
ax3.plot(times-times[-1], dxp1/max(dxp1), label="all bunches")

ax3.plot(times_reg-times[-1], dxp_mt/max(dxp1), label="mt bunches")

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

ValueError: x and y must have same first dimension

In [35]:
times_reg.shape, dxp_mt.shape

AttributeError: 'list' object has no attribute 'shape'

#### Multi turn test
---

In [30]:
slicer = UniformBinSlicer(500, z_cuts=(-5, 0.5))

bunches_source = sum(bunches_list[:3])
bunches_target = sum(bunches_list[3:])
bunches_target.dt = 0

s_source = bunches_source.get_slices(slicer)
s_target = bunches_target.get_slices(slicer)

n_turns, n_bunches = 2, 1
times_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))
cmoments_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))

times_reg[0,0,:] = s_target.z_centers/(beta*c)
times_reg[1,0,:] = s_source.z_centers/(beta*c)
cmoments_reg[0,0,:] = s_target.charge_per_slice
cmoments_reg[1,0,:] = s_source.charge_per_slice

ages = [0, T0]
betas = [bunches_target.beta, bunches_source.beta]

# slices = []
# for i in range(n_turns):
#     for j in range(n_bunches):
#         slices.append(bunches_mt[i,j].get_slices(slicer))
        
# slices = np.reshape(slices, (n_turns, n_bunches))



In [31]:
dxp_mt = convolve_PyHEADTAIL_mb(bunches_target, times_reg, ages, cmoments_reg, betas)

In [138]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

#### Multiple bunches

In [24]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

times_list, ages, cmoments_list, betas = get_times_and_moments(bunches_list, slicer)

delta_xp_mb_l = convolve_PyHEADTAIL_mb(bunches_list, times_list, ages, cmoments_list, betas)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)

delta_xp_sb_l = []
delta_xp_np_l = []
for i, b in enumerate(bunches_list):
    delta_xp_sb_l.append(convolve_PyHEADTAIL_sb(
            b, times_list[:,i,:], ages, cmoments_list[:,i,:], betas))
    delta_xp_np_l.append(convolve_multibunch(
            times_list[:,i,:], cmoments_list[:,i,:], wakefunction, f_convolution=convolution_numpy))

In [26]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

dxp1_l = np.squeeze(delta_xp_mb_l)
dxp2_l = np.squeeze(delta_xp_sb_l)
dxp3_l = np.squeeze(delta_xp_np_l)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 7)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments), '--')
ax3.plot(times-times[-1], dxp1/max(dxp1), '--', label="all bunches")

labels = ["first bunch", "second bunch", "third bunch"]
for i in xrange(len(bunches_list)):
    tml = np.squeeze(times_list[:,i,:])
    cml = np.squeeze(cmoments_list[:,i,:])
    ax3.plot(tml-times[-1]-dt[i], 2*cml/np.max(cmoments_list), c=col[i], label=labels[i])
    ax3.plot(tml-times[-1]-dt[i], dxp1_l[i]/max(dxp1), c=col[i+3])

#     [ax3.axvline(s, c=c[0], alpha=0.8) for s in (dts-dts[-1])]
#     [ax3.axvline(s, c=c[1], alpha=0.8) for s in (dts-dts[-1]-dt[i])]
#     [ax3.axvline(s, c=c[2], alpha=0.8) for s in (alltimes-alltimes[-1])]

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

(-2.1, 2.1)

In [21]:
plt.close('all')